In [1]:
from dyeles import DyeLeS
from rdkit import Chem
from pathlib import Path
import pandas as pd

/home/water/miniforge3/envs/DyeLeS/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = Path("../data")

coconut_df = pd.read_csv(data_dir / "processed" / "coconut.csv")
dyes_df = pd.read_csv(data_dir / "processed" / "dyes.csv")
zinc_df = pd.read_csv(data_dir / "processed" / "zinc.csv")

In [3]:
def build_validation_set(pos_df, neg_df):
    pos_df = pos_df[["smiles"]]
    neg_df = neg_df[["smiles"]]
    min_len = min(len(pos_df), len(neg_df))
    pos_df = pos_df.sample(min_len, random_state=42)
    neg_df = neg_df.sample(min_len, random_state=42)
    pos_df["label"] = 1
    neg_df["label"] = 0
    return pd.concat([pos_df, neg_df])

In [6]:
dyes_zinc_df = build_validation_set(dyes_df, zinc_df)
dyes_coconut_df = build_validation_set(dyes_df, coconut_df)

In [ ]:
scorer = DyeLeS()

def plot_auc_roc(df):
    y_true = df["smiles"].to_list()
    y_pred = scorer.score_batch(df["smiles"])
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)
    return roc_auc